In [22]:
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers.recurrent import LSTM

from data.models import SentenceBatch, Task
from keras_ml.models import Model, TrainSession
from word_embeddings.utils import load_w2v_model
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

from tqdm import tqdm_notebook
import itertools

In [23]:
ASPECT_ENTITY = 'RESTAURANT'
ASPECT_ATTRIBUTE = 'MISCELLANEOUS'

In [24]:
task = Task.objects.get(
    type=Task.Type.ASPECT_DETECTION,
    aspect_entity=ASPECT_ENTITY,
    aspect_attribute=ASPECT_ATTRIBUTE
)
batch = SentenceBatch.objects.filter(task=task).first()
w2v_model = load_w2v_model(batch.w2v_model)

x_train = np.array(batch.x_train)
y_train = np.array(batch.y_train)

Loading w2v model using cache


In [25]:
def create_model():
    embedding_layer = Embedding(
    input_dim=len(w2v_model.wv.vocab),
    output_dim=w2v_model.vector_size,
    input_length=len(batch.x_train[0]),
    mask_zero=True
    )

    lstm_layer = LSTM(
        units=200,
        activation='selu',
        recurrent_activation='hard_sigmoid'
    )
    dense_layer = Dense(1, activation='sigmoid')

    model = Sequential()
    model.add(embedding_layer)
    model.add(lstm_layer)
    model.add(dense_layer)

    compile_opts = dict(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    
#     train_opts = dict(
#         epochs=10,
#         validation_split=0.1,
#         batch_size=128
#     )

    model.compile(**compile_opts)
    return model
    

In [37]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100, 128]
epochs = [10, 50, 100]
validation_split=[0.1]

In [38]:
param_grid = dict(batch_size=batch_size, epochs=epochs, validation_split=validation_split)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train, y_train, verbose=0)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: 